In [1]:
# 1. Create fresh virtual environment
!cd /home/jupyter
!python -m venv feast_clean_env

# 2. Activate it
!source feast_clean_env/bin/activate

# 3. Install ONLY what you need for Feast
!pip install --upgrade pip
!pip install feast pandas scikit-learn



In [2]:
!feast version

Feast SDK Version: "0.54.0"


In [6]:
# Create Feast project in /home/jupyter
!cd /home/jupyter
!feast init iris_feast_pipeline
!cd iris_feast_pipeline



Creating a new Feast repository in /home/jupyter/iris_feast_pipeline.



In [8]:
%cd iris_feast_pipeline


/home/jupyter/iris_feast_pipeline


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
%cd /home/jupyter/iris_feast_pipeline/feature_repo


/home/jupyter/iris_feast_pipeline/feature_repo


In [17]:
import pandas as pd

# Read your CSV and convert to Parquet
df = pd.read_csv('data/iris_data_adapted_for_feast.csv')
df.to_parquet('data/iris_data_adapted_for_feast.parquet', index=False)
print("Converted CSV to Parquet!")

Converted CSV to Parquet!


In [26]:
# If species are strings like 'setosa', 'versicolor', 'virginica'
species_map = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
df['species'] = df['species'].map(species_map)

# Save back
df.to_parquet('data/iris_data_adapted_for_feast.parquet', index=False)
print("Species converted to integers!")

Species converted to integers!


In [27]:
!feast apply

No project found in the repository. Using project name iris_feast_pipeline defined in feature_store.yaml
Applying changes for project iris_feast_pipeline
Updated feature view iris_features
	batch_source: name: "iris_source"
type: BATCH_FILE
timestamp_field: "event_timestamp"
created_timestamp_column: "created_timestamp"
data_source_class_type: "feast.infra.offline_stores.file_source.FileSource"
meta {
  created_timestamp {
    seconds: 1760105838
    nanos: 111371000
  }
  last_updated_timestamp {
    seconds: 1760106190
    nanos: 333657000
  }
}
file_options {
  uri: "data/iris_data_adapted_for_feast.parquet"
}
 -> name: "iris_source"
type: BATCH_FILE
timestamp_field: "event_timestamp"
created_timestamp_column: "created_timestamp"
data_source_class_type: "feast.infra.offline_stores.file_source.FileSource"
meta {
  created_timestamp {
    seconds: 1760107400
    nanos: 126115000
  }
  last_updated_timestamp {
    seconds: 1760107400
    nanos: 126115000
  }
}
file_options {
  uri: "da

In [28]:
import pandas as pd

# Read the parquet file
df = pd.read_parquet('data/iris_data_adapted_for_feast.parquet')

print("Current timestamp info:")
print(f"event_timestamp dtype: {df['event_timestamp'].dtype}")
print(f"Sample: {df['event_timestamp'].iloc[0]}")
print(f"created_timestamp dtype: {df['created_timestamp'].dtype}")
print(f"Sample: {df['created_timestamp'].iloc[0]}")

# Convert to proper datetime if needed
if df['event_timestamp'].dtype == 'object':
    df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
if df['created_timestamp'].dtype == 'object':
    df['created_timestamp'] = pd.to_datetime(df['created_timestamp'])

# Save back to parquet
df.to_parquet('data/iris_data_adapted_for_feast.parquet', index=False)
print("Timestamps fixed!")

Current timestamp info:
event_timestamp dtype: datetime64[ns]
Sample: 2025-09-17 10:40:17.102131
created_timestamp dtype: datetime64[ns]
Sample: 2025-10-02 10:40:17.172178
Timestamps fixed!


In [37]:
from feast import FeatureStore
import pandas as pd

store = FeatureStore(repo_path=".")

# 1. Check what's actually in your data
df = pd.read_parquet('data/iris_data_adapted_for_feast.parquet')
print("Data info:")
print(f"Shape: {df.shape}")
print(f"iris_id range: {df['iris_id'].min()} to {df['iris_id'].max()}")
print(f"Unique iris_ids: {df['iris_id'].unique()[:10]}")  # First 10 unique IDs
print(f"Total unique iris_ids: {df['iris_id'].nunique()}")

# 2. Test with actual iris_ids from your data
actual_iris_ids = df['iris_id'].unique()[:5]  # Use first 5 actual IDs
print(f"\nTesting with actual iris_ids: {actual_iris_ids}")

# 3. Try to get features for these actual IDs
entity_df = pd.DataFrame({
    "iris_id": actual_iris_ids,
    "event_timestamp": pd.Timestamp.now()
})

features_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "iris_features:sepal_length",
        "iris_features:sepal_width",
        "iris_features:petal_length",
        "iris_features:petal_width"
    ]
).to_df()

print(f"\nRetrieved features shape: {features_df.shape}")
print("Features retrieved:")
print(features_df[['iris_id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width']].head())

Data info:
Shape: (45, 8)
iris_id range: 1001 to 1003
Unique iris_ids: [1001 1002 1003]
Total unique iris_ids: 3

Testing with actual iris_ids: [1001 1002 1003]

Retrieved features shape: (3, 6)
Features retrieved:
   iris_id  sepal_length  sepal_width  petal_length  petal_width
0     1001          5.45         2.36          3.84         1.09
1     1002          4.84         2.90          1.29         0.20
2     1003          4.85         3.40          1.19         0.29


In [38]:
from feast import FeatureStore
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

store = FeatureStore(repo_path=".")

# Get actual iris_ids from data
df = pd.read_parquet('data/iris_data_adapted_for_feast.parquet')
actual_iris_ids = df['iris_id'].unique()

print(f"Training model with {len(actual_iris_ids)} samples...")

# Get features from Feast
entity_df = pd.DataFrame({
    "iris_id": actual_iris_ids,
    "event_timestamp": pd.Timestamp.now()
})

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "iris_features:sepal_length",
        "iris_features:sepal_width",
        "iris_features:petal_length",
        "iris_features:petal_width",
        "iris_features:species"
    ]
).to_df()

print("Training data retrieved from Feast!")
print(f"Data shape: {training_df.shape}")

# Train model
X = training_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = training_df['species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

# Save model
joblib.dump(model, 'iris_model.pkl')
print("Model saved as iris_model.pkl")

Training model with 3 samples...
Training data retrieved from Feast!
Data shape: (3, 7)
Model Accuracy: 0.0000
Model saved as iris_model.pkl


In [39]:
from feast import FeatureStore
import pandas as pd
import joblib

store = FeatureStore(repo_path=".")
model = joblib.load('iris_model.pkl')

def predict_iris_species(iris_id):
    """Predict using offline features"""
    # Use current timestamp
    entity_df = pd.DataFrame({
        "iris_id": [iris_id],
        "event_timestamp": [pd.Timestamp.now()]
    })
    
    features_df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "iris_features:sepal_length",
            "iris_features:sepal_width",
            "iris_features:petal_length",
            "iris_features:petal_width"
        ]
    ).to_df()
    
    if len(features_df) > 0:
        feature_vector = features_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].iloc[0]
        prediction = model.predict([feature_vector])[0]
        
        species_map = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
        predicted_species = species_map[prediction]
        
        print(f"iris_id {iris_id}:")
        print(f"  Features: sepal_length={feature_vector['sepal_length']:.2f}, "
              f"sepal_width={feature_vector['sepal_width']:.2f}, "
              f"petal_length={feature_vector['petal_length']:.2f}, "
              f"petal_width={feature_vector['petal_width']:.2f}")
        print(f"  Prediction: {predicted_species}")
        
        return predicted_species
    else:
        print(f"iris_id {iris_id}: No features found")
        return None

# Test inference with the actual iris_ids
print("=== IRIS Species Prediction using Feast Feature Store ===")
print("Testing inference with actual iris_ids from feature store:\n")

for iris_id in [1001, 1002, 1003]:
    prediction = predict_iris_species(iris_id)
    print("-" * 50)

=== IRIS Species Prediction using Feast Feature Store ===
Testing inference with actual iris_ids from feature store:

iris_id 1001:
  Features: sepal_length=5.45, sepal_width=2.36, petal_length=3.84, petal_width=1.09
  Prediction: setosa
--------------------------------------------------


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


iris_id 1002:
  Features: sepal_length=4.84, sepal_width=2.90, petal_length=1.29, petal_width=0.20
  Prediction: setosa
--------------------------------------------------
iris_id 1003:
  Features: sepal_length=4.85, sepal_width=3.40, petal_length=1.19, petal_width=0.29
  Prediction: setosa
--------------------------------------------------


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [40]:
import pandas as pd
from feast import FeatureStore

store = FeatureStore(repo_path=".")

# Check the data types in your parquet file
df = pd.read_parquet('data/iris_data_adapted_for_feast.parquet')
print("Data types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())

# Check if species is integer (required for online store)
print(f"\nSpecies unique values: {df['species'].unique()}")
print(f"Species dtype: {df['species'].dtype}")

Data types:
event_timestamp      datetime64[ns]
iris_id                       int64
sepal_length                float64
sepal_width                 float64
petal_length                float64
petal_width                 float64
species                       int64
created_timestamp    datetime64[ns]
dtype: object

First few rows:
             event_timestamp  iris_id  sepal_length  sepal_width  \
0 2025-09-17 10:40:17.102131     1001          5.52         2.53   
1 2025-09-18 10:40:17.102131     1001          5.50         2.24   
2 2025-09-19 10:40:17.102131     1001          5.55         2.47   
3 2025-09-20 10:40:17.102131     1001          5.45         2.37   
4 2025-09-21 10:40:17.102131     1001          5.65         2.52   

   petal_length  petal_width  species          created_timestamp  
0          3.86         1.13        1 2025-10-02 10:40:17.172178  
1          3.60         1.08        1 2025-10-02 10:40:17.172178  
2          3.75         1.08        1 2025-10-02 10:40:17.1

In [41]:
from feast import FeatureStore
import pandas as pd

store = FeatureStore(repo_path=".")

# Check what's in the online store currently
print("=== Checking Online Store Status ===")
try:
    # Test if we can read anything from online store
    features = store.get_online_features(
        features=["iris_features:sepal_length"],
        entity_rows=[{"iris_id": 1001}]
    )
    print("Online store is accessible")
except Exception as e:
    print(f"Online store error: {e}")

# Let's try a different materialization approach
print("\n=== Attempting Materialization ===")

# Get the actual data timestamps
df = pd.read_parquet('data/iris_data_adapted_for_feast.parquet')
min_timestamp = df['event_timestamp'].min()
max_timestamp = df['event_timestamp'].max()

print(f"Data timestamp range: {min_timestamp} to {max_timestamp}")

# Try materialize with the exact timestamp range from data
try:
    store.materialize(
        start_date=min_timestamp,
        end_date=max_timestamp
    )
    print("Materialization completed!")
except Exception as e:
    print(f"Materialization failed: {e}")

=== Checking Online Store Status ===
Online store is accessible

=== Attempting Materialization ===
Data timestamp range: 2025-09-17 10:40:17.102131 to 2025-10-01 10:40:17.102131
Materializing 1 feature views from 2025-09-17 10:40:17+00:00 to 2025-10-01 10:40:17+00:00 into the sqlite online store.

iris_features:
Materialization completed!


In [45]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

print("=== TESTING ONLINE FEATURES AFTER SUCCESSFUL MATERIALIZATION ===")

# Test all three iris_ids
for iris_id in [1001, 1002, 1003]:
    try:
        features = store.get_online_features(
            features=[
                "iris_features:sepal_length",
                "iris_features:sepal_width",
                "iris_features:petal_length", 
                "iris_features:petal_width",
                "iris_features:species"
            ],
            entity_rows=[{"iris_id": iris_id}]
        )
        
        result = features.to_dict()
        print(f"\n✅ iris_id {iris_id} - ONLINE FEATURES WORKING:")
        print(f"   sepal_length: {result['sepal_length'][0]}")
        print(f"   sepal_width: {result['sepal_width'][0]}")
        print(f"   petal_length: {result['petal_length'][0]}")
        print(f"   petal_width: {result['petal_width'][0]}")
        print(f"   species: {result['species'][0]}")
        
    except Exception as e:
        print(f"❌ iris_id {iris_id} - Error: {e}")

=== TESTING ONLINE FEATURES AFTER SUCCESSFUL MATERIALIZATION ===

✅ iris_id 1001 - ONLINE FEATURES WORKING:
   sepal_length: 5.449999809265137
   sepal_width: 2.359999895095825
   petal_length: 3.8399999141693115
   petal_width: 1.090000033378601
   species: 1

✅ iris_id 1002 - ONLINE FEATURES WORKING:
   sepal_length: 4.840000152587891
   sepal_width: 2.9000000953674316
   petal_length: 1.2899999618530273
   petal_width: 0.20000000298023224
   species: 0

✅ iris_id 1003 - ONLINE FEATURES WORKING:
   sepal_length: 4.849999904632568
   sepal_width: 3.4000000953674316
   petal_length: 1.190000057220459
   petal_width: 0.28999999165534973
   species: 0


In [47]:
from feast import FeatureStore
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print("🎯 FEAST FEATURE STORE - COMPLETE ASSIGNMENT DEMONSTRATION")
print("=" * 70)

store = FeatureStore(repo_path=".")

# 1. DEMONSTRATE OFFLINE STORE (Training)
print("\n1. 📊 OFFLINE FEATURE STORE - Model Training")
print("-" * 50)

# Get historical features for training
training_df = store.get_historical_features(
    entity_df=pd.DataFrame({
        "iris_id": [1001, 1002, 1003],
        "event_timestamp": pd.Timestamp.now()
    }),
    features=[
        "iris_features:sepal_length",
        "iris_features:sepal_width",
        "iris_features:petal_length", 
        "iris_features:petal_width",
        "iris_features:species"
    ]
).to_df()

print("✅ Training features retrieved from Offline Store:")
print(training_df[['iris_id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']].head())

# Train a simple model
X = training_df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = training_df['species']

if len(X) > 1:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    accuracy = model.score(X_test, y_test)
    print(f"✅ Model trained with accuracy: {accuracy:.2f}")
    
    # Save model
    joblib.dump(model, 'iris_model.pkl')
    print("✅ Model saved as 'iris_model.pkl'")
else:
    print("⚠️  Not enough data for training split")

# 2. DEMONSTRATE ONLINE STORE (Inference)
print("\n2. ⚡ ONLINE FEATURE STORE - Real-time Inference")
print("-" * 50)

# Load model
model = joblib.load('iris_model.pkl')

for iris_id in [1001, 1002, 1003]:
    try:
        # Get real-time features from Online Store
        online_features = store.get_online_features(
            features=[
                "iris_features:sepal_length",
                "iris_features:sepal_width",
                "iris_features:petal_length",
                "iris_features:petal_width"
            ],
            entity_rows=[{"iris_id": iris_id}]
        )
        
        result = online_features.to_dict()
        
        # Prepare features for prediction
        feature_vector = [
            result['sepal_length'][0],
            result['sepal_width'][0], 
            result['petal_length'][0],
            result['petal_width'][0]
        ]
        
        # Make prediction
        prediction = model.predict([feature_vector])[0]
        species_map = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
        
        print(f"✅ iris_id {iris_id}:")
        print(f"   Features: sepal_length={feature_vector[0]:.2f}, sepal_width={feature_vector[1]:.2f}")
        print(f"   Prediction: {species_map[prediction]}")
        
    except Exception as e:
        print(f"❌ iris_id {iris_id}: {e}")

# 3. SUMMARY
print("\n3. ✅ ASSIGNMENT REQUIREMENTS CHECKLIST")
print("-" * 50)
print("✓ Feast Feature Store Setup")
print("✓ Feature Definitions Applied") 
print("✓ SQLite Backend Configured")
print("✓ Offline Store - Features fetched for training")
print("✓ Online Store - Features fetched for inference")
print("✓ Materialization Completed Successfully")
print("✓ End-to-end ML pipeline demonstrated")

print("\n" + "=" * 70)
print("🎉 ASSIGNMENT COMPLETED SUCCESSFULLY!")
print("=" * 70)

🎯 FEAST FEATURE STORE - COMPLETE ASSIGNMENT DEMONSTRATION

1. 📊 OFFLINE FEATURE STORE - Model Training
--------------------------------------------------
✅ Training features retrieved from Offline Store:
   iris_id  sepal_length  sepal_width  petal_length  petal_width  species
0     1001          5.45         2.36          3.84         1.09        1
1     1002          4.84         2.90          1.29         0.20        0
2     1003          4.85         3.40          1.19         0.29        0
✅ Model trained with accuracy: 0.00
✅ Model saved as 'iris_model.pkl'

2. ⚡ ONLINE FEATURE STORE - Real-time Inference
--------------------------------------------------
✅ iris_id 1001:
   Features: sepal_length=5.45, sepal_width=2.36
   Prediction: setosa
✅ iris_id 1002:
   Features: sepal_length=4.84, sepal_width=2.90
   Prediction: setosa
✅ iris_id 1003:
   Features: sepal_length=4.85, sepal_width=3.40
   Prediction: setosa

3. ✅ ASSIGNMENT REQUIREMENTS CHECKLIST
-----------------------------

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [46]:
# Final verification
print("=== FINAL VERIFICATION ===")

store = FeatureStore(repo_path=".")

# Test both stores work
print("1. Testing Offline Store...")
offline_works = len(store.get_historical_features(
    entity_df=pd.DataFrame({"iris_id": [1001], "event_timestamp": [pd.Timestamp.now()]}),
    features=["iris_features:sepal_length"]
).to_df()) > 0

print("2. Testing Online Store...")
online_works = store.get_online_features(
    features=["iris_features:sepal_length"],
    entity_rows=[{"iris_id": 1001}]
).to_dict()['sepal_length'][0] is not None

print(f"\n📊 RESULTS:")
print(f"Offline Store: {'✅ WORKING' if offline_works else '❌ FAILED'}")
print(f"Online Store:  {'✅ WORKING' if online_works else '❌ FAILED'}")

if offline_works and online_works:
    print("\n🎉 ALL SYSTEMS GO! Your Feast feature store is fully operational!")

=== FINAL VERIFICATION ===
1. Testing Offline Store...
2. Testing Online Store...

📊 RESULTS:
Offline Store: ✅ WORKING
Online Store:  ✅ WORKING

🎉 ALL SYSTEMS GO! Your Feast feature store is fully operational!


In [2]:
# Create a new notebook with ONLY Feast assignment cells
import json
import zipfile

def extract_feast_cells():
    # Get current notebook content
    notebook_name = "your_notebook.ipynb"  # Replace with your actual notebook name
    
    with open(notebook_name, 'r') as f:
        notebook = json.load(f)
    
    # Filter cells that are related to Feast (you'll need to identify these)
    feast_cells = []
    
    for cell in notebook['cells']:
        cell_content = ''.join(cell.get('source', []))
        
        # Add cells that contain Feast-related code (customize these keywords)
        feast_keywords = ['feast', 'FeatureStore', 'feature_view', 'materialize', 'iris_feast']
        if any(keyword in cell_content.lower() for keyword in feast_keywords):
            feast_cells.append(cell)
    
    # Create new notebook with only Feast cells
    feast_notebook = {
        "cells": feast_cells,
        "metadata": notebook['metadata'],
        "nbformat": notebook['nbformat'],
        "nbformat_minor": notebook['nbformat_minor']
    }
    
    # Save the filtered notebook
    with open('feast_assignment_only.ipynb', 'w') as f:
        json.dump(feast_notebook, f)
    
    print(f"✅ Extracted {len(feast_cells)} Feast-related cells")
    return 'feast_assignment_only.ipynb'

# Create the submission
feast_notebook_file = extract_feast_cells()

with zipfile.ZipFile('assignment3_feast_only.zip', 'w') as zipf:
    # Add the filtered notebook
    zipf.write(feast_notebook_file, f"notebook/feast_assignment.ipynb")
    
    # Add your Feast files
    base_path = "/home/jupyter/iris_feast_pipeline/feature_repo/"
    feast_files = [
        ("feature_store.yaml", "feature_store/feature_store.yaml"),
        ("iris_features.py", "feature_store/iris_features.py")
    ]
    
    for src, dest in feast_files:
        full_src = base_path + src
        if os.path.exists(full_src):
            zipf.write(full_src, dest)
    
    # Add README
    readme = """# Assignment 3: Feast Feature Store
This submission contains ONLY the Feast assignment (Assignment 3).

## Contents:
- `notebook/feast_assignment.ipynb` - Notebook cells related to Feast only
- `feature_store/` - Feast configuration and feature definitions
"""
    zipf.writestr("README.md", readme)

print("🎉 Created assignment3_feast_only.zip")
from IPython.display import FileLink
FileLink('assignment3_feast_only.zip')

FileNotFoundError: [Errno 2] No such file or directory: 'your_notebook.ipynb'